In [22]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
Fp = GF(p) # the base field of JubJub

In [23]:
# See details here: 
# https://www.ietf.org/archive/id/draft-irtf-cfrg-hash-to-curve-10.html#appx-rational-map




Given the equation in Montgomery form:
$$ K \cdot t^2 = s^3 + J \cdot s^2 + s $$
return the corresponding Weierstrass form:
$$y^2 = x^3 + A \cdot x + B$$
where:
$$ A = \frac{3 - J^2}{3 \cdot K^2} $$
$$ B = \frac{2 \cdot J^3 - 9 \cdot J}{27 \cdot K^3} $$



In [24]:
def montgomery_to_weierstrass(K, J):
    A = (3 - J * J) / (3 * K * K)
    B = (2 * J * J * J - 9 * J) / (27 * K * K * K)
    return (A, B)

In [25]:
(A, B) = montgomery_to_weierstrass(1, 168698)
(A, B)

(-28459015201/3, 9601957892250502/27)

In [26]:
A = int(Fp(A))
B = int(Fp(B))

In [27]:
# https://www.rfc-editor.org/rfc/rfc9380.html#svdw-z-code
# Arguments:
# - F, a field object, e.g., F = GF(2^521 - 1)
# - A and B, the coefficients of the curve y^2 = x^3 + A * x + B
def find_z_svdw(F, A, B, init_ctr=1):
    g = lambda x: F(x)^3 + F(A) * F(x) + F(B)
    h = lambda Z: -(F(3) * Z^2 + F(4) * A) / (F(4) * g(Z))
    # NOTE: if init_ctr=1 fails to find Z, try setting it to F.gen()
    ctr = init_ctr
    while True:
        for Z_cand in (F(ctr), F(-ctr)):
            # Criterion 1:
            #   g(Z) != 0 in F.
            if g(Z_cand) == F(0):
                continue
            # Criterion 2:
            #   -(3 * Z^2 + 4 * A) / (4 * g(Z)) != 0 in F.
            if h(Z_cand) == F(0):
                continue
            # Criterion 3:
            #   -(3 * Z^2 + 4 * A) / (4 * g(Z)) is square in F.
            if not is_square(h(Z_cand)):
                continue
            # Criterion 4:
            #   At least one of g(Z) and g(-Z / 2) is square in F.
            if is_square(g(Z_cand)) or is_square(g(-Z_cand / F(2))):
                return Z_cand
        ctr += 1

In [28]:
# Given an integer with 256 bits in hex, convert it into 4 limbs of 64 bits where the least significent limb is put first.

def u_64_little_endian(n):
    str_hex = n
    str_hex_without_0x = str_hex[2:]
    # pad 0s into hex litteral of 64 digits for the 256 bits 
    full_width_str = '0' * (64 - len(str_hex_without_0x)) + str_hex_without_0x
    assert len(full_width_str) == 64

    res = []
    # divide the litteral into 4 limbs of 64 bits, the least significent limb is put first
    for i in range(4):
        temp = '0x' + full_width_str[64 - 16 * (i + 1) : 64 - 16 * i]
        res.append(temp)
    return res

def print_for_rust(elem):
    print( hex(elem))
    print('\n')
    print('[')
    for limb in u_64_little_endian(elem):
        print(limb + ",")
    print(']')
    print('------\n')

In [29]:
jubjub_z = find_z_svdw(Fp, A, B)
jubjub_z

1

In [30]:
kronecker(7, p) # i.e 7 is a quadratic non-residue

-1